In [8]:
import os
import glob
import pandas as pd
def get_data(dn):
    data = {
        "news":[],
        "ans":[]
    }
    dirs = glob.glob(os.path.join(dn, "*"))
    for d in dirs:
        pat = os.path.join(d, "*.[tT][xX][tT]")
        fs = glob.glob(pat)
        for fp in fs:
            with open(fp, "r", encoding="utf-8") as f:
                news = f.read()
            ans = os.path.split(d)[-1]
            data["news"].append(news)
            data["ans"].append(ans)
    df = pd.DataFrame(data, columns=["news", "ans"])
    return df

In [10]:
train_df = get_data("chinese_news_trans/chinese_news_trans")
test_df = get_data("chinese_news_test/chinese_news_test")
test_df

,news,ans
0,日月光華 -- Traffic_Info精華區文章閱讀\n-----------------...,交通
1,日月光華 -- Traffic_Info精華區文章閱讀\n-----------------...,交通
2,日月光華 -- Traffic_Info精華區文章閱讀\n-----------------...,交通
3,三趟火車停開 乘客可全額退票 \n瀏覽次數：1180 \n 昨日，來自鐵路部門的消息說...,交通
4,日月光華 -- Traffic_Info精華區文章閱讀\n-----------------...,交通
...,...,...
96,最優秀選手無緣亞運會健美賽\n \n \n 健美在亞洲運動會上是“新生兒”——韓國釜...,體育
97,\n各國記者眼中的羽毛球世錦賽\n \n\n-----------------------...,體育
98,\n友好運動會第五天 東道主選手大顯神威\n2001年09月03日02:51:40 新華社 ...,體育
99,不靠技術比運氣\n\n 第二屆奧運會在巴黎舉行，同時這裡也正在舉行國際博覽會，東道主把一些...,體育


In [20]:
# unique value_counts
kind = train_df["ans"].unique()
# enumerate
trans = {k:i for i, k in enumerate(kind)}
reverse_trans = {i:k for i, k in enumerate(kind)}

In [22]:
y_train = train_df["ans"].replace(trans)
y_test = test_df["ans"].replace(trans)

In [24]:
import os
from urllib.request import urlretrieve
DICT_PATH = "dict.txt.big"
if not os.path.exists(DICT_PATH):
    print("字典不存在")
    url = "https://github.com/fxsjy/jieba/raw/master/extra_dict/dict.txt.big"
    urlretrieve(url, DICT_PATH)

In [26]:
import jieba
jieba.set_dictionary(DICT_PATH)
def newscut(news):
    s = " ".join(jieba.cut(news))
    return s.replace("\r", "").replace("\n", "")
x_train_raw = train_df["news"].apply(newscut)
x_test_raw = test_df["news"].apply(newscut)

Building prefix dict from C:\Users\user\PycharmProjects\MLDemo\dict.txt.big ...
Loading model from cache C:\Users\user\AppData\Local\Temp\jieba.u8b0aede1d05d994ba4294f10e50c49e7.cache
Loading model cost 1.291 seconds.
Prefix dict has been built successfully.


In [32]:
from sklearn.feature_extraction.text import CountVectorizer
vec = CountVectorizer()
x_train = vec.fit_transform(x_train_raw)
x_test = vec.transform(x_test_raw)

In [33]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf.fit(x_train, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [34]:
from sklearn.metrics import accuracy_score
pre = clf.predict(x_test)
accuracy_score(pre, y_test)

1.0

In [35]:
from sklearn.metrics import confusion_matrix
mat = confusion_matrix(y_test, pre)
col = ["{}(預測)".format(k) for k in kind]
row = ["{}(答案)".format(k) for k in kind]
pd.DataFrame(mat, columns=col, index=row)

,交通(預測),政治(預測),教育(預測),環境(預測),經濟(預測),藝術(預測),計算機(預測),軍事(預測),醫藥(預測),體育(預測)
交通(答案),10,0,0,0,0,0,0,0,0,0
政治(答案),0,11,0,0,0,0,0,0,0,0
教育(答案),0,0,10,0,0,0,0,0,0,0
環境(答案),0,0,0,10,0,0,0,0,0,0
經濟(答案),0,0,0,0,10,0,0,0,0,0
藝術(答案),0,0,0,0,0,10,0,0,0,0
計算機(答案),0,0,0,0,0,0,10,0,0,0
軍事(答案),0,0,0,0,0,0,0,10,0,0
醫藥(答案),0,0,0,0,0,0,0,0,10,0
體育(答案),0,0,0,0,0,0,0,0,0,10


In [45]:
n = input("請輸入新聞:")
test = vec.transform([newscut(n)])
pre = clf.predict(test)[0]
print("應該是:", reverse_trans[pre])
proba = clf.predict_proba(test)[0]
proba = list(enumerate(proba))
for i, prob in sorted(proba, key=lambda x:x[1], reverse=True):
    print(reverse_trans[i], ":", prob * 100, "%")

請輸入新聞:Sony Interactive Entertainment在官方公告中提到，有鑑人們為防止肺炎疫情擴散安分守己地待在家中，為了感謝大家保持社交距離，並盡量滿足大家「Play At Home」（在家遊玩）的需求，因此決定推出包含兩部分內容的「Play At Home」活動，第一是提供給所有PS4的玩家在限定期間內可免費下載《秘境探險：德瑞克合輯》與《風之旅人》2款遊戲，兌換後即可一直擁有。     《秘境探險：德瑞克合輯(UNCHARTED: The Nathan Drake Collection)》，是將原本由遊戲製作室Naughty Dog創作的冒險遊戲，由Bluepoint Games發揮PS4™主機的性能重製登場。合輯收錄了系列前 3 代《秘境探險：黃金城秘寶》、《秘境探險 2：盜亦有道》、《秘境探險 3：德瑞克的騙局》的遊戲，玩家將扮演奈森・德瑞克，從微不足道的事端開始，到難以想像的發現，踏上橫越地球的驚險旅程。而德瑞克亦會與令人難忘的各式人物相遇，以性命和友情作為賭注，與陰險殘酷的敵人競爭追蹤傳說中的寶藏。目前PS Store售價為NT$590元，並曾作為2020年1月「PlayStation Plus」會員免費下載遊戲。
應該是: 計算機
計算機 : 100.0 %
藝術 : 3.218748541262935e-26 %
軍事 : 3.022596169929378e-26 %
體育 : 6.990167772034091e-28 %
經濟 : 3.6804377404517384e-30 %
環境 : 1.2686136324264227e-31 %
教育 : 1.9778955176486755e-33 %
醫藥 : 7.520885412500497e-34 %
交通 : 9.988705488913441e-35 %
政治 : 3.0765746621197374e-40 %


In [40]:
list(zip([1, 2, 3],
         [7, 8, 9],
         [4, 5, 6]))

[(1, 7, 4), (2, 8, 5), (3, 9, 6)]

In [41]:
sorted([(3, "a"), (1, "b"), (5, "c")])

[(1, 'b'), (3, 'a'), (5, 'c')]